In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.layers import Input, Dense, Dropout, LeakyReLU
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

In [2]:
np.random.seed(10)
random_dim=100

In [5]:
def load_mnist_dat():
  (x_train,y_train),(x_test,y_test)=mnist.load_data()
  x_train=(x_train.astype(np.float32) - 127.5)/127.5
  x_train=x_train.reshape(60000, 784)
  return(x_train,y_train,x_test,y_test)

In [6]:
def get_optimizer():
  return Adam(learning_rate=0.00002,beta_1=0.5)

In [16]:
def get_generator(optimizer):
  generator = Sequential()
  generator.add(Dense(256, input_dim=random_dim,\
                      kernel_initializer=initializers.RandomNormal (stddev=0.02)))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense (784, activation='tanh'))
  generator.compile(loss='binary_crossentropy', optimizer= optimizer)
  return generator

In [20]:
def get_discriminator(optimizer):
  discriminator = Sequential()
  discriminator.add(Dense (1024, input_dim=784,\
                           kernel_initializer=initializers.RandomNormal (stddev=0.02)))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(1, activation='sigmoid'))
  discriminator.compile(loss='binary_crossentropy', optimizer =optimizer)
  return discriminator

In [21]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
  discriminator.trainable = False
  discriminator.trainable=False
  gan_input = Input(shape=(random_dim,))
  x = generator(gan_input)
  gan_output = discriminator(x)
  gan = Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer=optimizer)
  return gan

In [25]:
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), \
                          figsize=(10, 10)):
  noise = np.random.normal(0, 1, size=[examples, random_dim])
  generated_images = generator.predict(noise)
  generated_images = generated_images.reshape(examples, 28, 28)
  plt.figure(figsize=figsize)
  for i in range(generated_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generated_images[i], interpolation='nearest',\
               cmap='gray_r')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)